<a href="https://colab.research.google.com/github/AbrahamKong/CMPE255-Apriori_and_FP_Growth/blob/main/CMPE_255_Apriori_and_FP_Growth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install apyori
!pip install mlxtend

  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=14e207008f21edc5f53c16fd3e6f0d2d051fc0207a6eb425cba042e76ef67bf4
  Stored in directory: /root/.cache/pip/wheels/cb/f6/e1/57973c631d27efd1a2f375bd6a83b2a616c4021f24aab84080
Successfully built apyori


In [2]:
# upgrade to current version, in case sometimes Google Colab has an older version

%pip install mlxtend --upgrade

     |████████████████████████████████| 1.3 MB 5.0 MB/s 
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0


In [ ]:
# import necessary library

import pandas as pd
import numpy as np
from apyori import apriori
from mlxtend.preprocessing import TransactionEncoder

In [ ]:
# Using data from earlier assignment in CMPE 256

df = pd.read_csv("https://ghcdn.rawgit.org/AbrahamKong/CMPE256-Apriori_Assignment/main/TRAIN-ARULES.csv")
df.head()

In [ ]:
# transform data set to meet the apriori function requirements

# Group data by order_id as instruction
# Transfer product names into a list for each transaction
df_grouped = df.groupby(by = ['order_id'])['product_name'].apply(list).reset_index(name='product_name')

In [ ]:
# Unpack list into their own column
df_grouped = df_grouped['product_name'].apply(pd.Series)

In [ ]:
# Trasform dataframe into a list to meet the apriori requirements
data = df_grouped.astype(str).values.tolist()

In [ ]:
# remove mull value from each row
data = [[ele for ele in sub if ele != 'nan'] for sub in data]

**Apriori Algorithm**

In [ ]:
# Apply apriori algorithm
association_rules = apriori(transactions = data, min_support=0.005) 
association_results = list(association_rules)
apriori_result = pd.DataFrame(association_results)

In [ ]:
apriori_result.head()

In [ ]:
apriori_result['ordered_statistics'][0]

**FP Growth**

In [ ]:
# instantiate a transaction encoder
my_transactionencoder = TransactionEncoder()

# fit the transaction encoder using the list of transaction tuples
my_transactionencoder.fit(transactions)

# transform the list of transaction tuples into an array of encoded transactions
encoded_transactions = my_transactionencoder.transform(transactions)

# convert the array of encoded transactions into a dataframe
encoded_transactions_df = pd.DataFrame(encoded_transactions, columns=my_transactionencoder.columns_)
encoded_transactions_df

In [ ]:
# our min support is 7, but it has to be expressed as a percentage for mlxtend
min_support = 7/len(transactions) 

# compute the frequent itemsets using fpgriowth from mlxtend
from mlxtend.frequent_patterns.fpgrowth import fpgrowth
frequent_itemsets = fpgrowth(encoded_transactions_df, min_support=min_support, use_colnames = True)

# print the frequent itemsets
frequent_itemsets

**Reference**


[The FP Growth algorithm](https://towardsdatascience.com/the-fp-growth-algorithm-1ffa20e839b8)